In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.13.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")

In [2]:
import com.fasterxml.jackson.annotation.JsonFormat
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import java.time.LocalDateTime
import java.util.ArrayList

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
}

data class Visit(
    val name: String,
    val location: Location,
    val load: Int
) {
    override fun toString(): String = name
}

@PlanningEntity
data class Vehicle(
    @PlanningId
    val name: String,
    val homeLocation: Location,
    val capacity: Int
) {
    @PlanningListVariable
    var visits: MutableList<Visit> = ArrayList()

    // No-arg constructor required for Timefold
    constructor() : this("", Location(0.0, 0.0), 0)

    override fun toString(): String = name
}

In [3]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider

class VehicleRoutingConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            capacity(constraintFactory),
            minimizeDistance(constraintFactory)
        )
    }

    fun capacity(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Vehicle::class.java)
            .filter { vehicle -> vehicle.visits.sumOf { it.load } > vehicle.capacity }
            .penalizeLong(HardSoftLongScore.ONE_HARD,
                { vehicle -> vehicle.visits.sumOf { it.load } - vehicle.capacity.toLong() })
            .asConstraint("vehicle-routing", "Capacity")
    }

    fun minimizeDistance(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Vehicle::class.java)
            .penalizeLong(HardSoftLongScore.ONE_SOFT, { vehicle ->
                var distance = 0.0
                var previousLocation = vehicle.homeLocation
                for (visit in vehicle.visits) {
                    distance += previousLocation.calcEuclideanDistanceTo(visit.location)
                    previousLocation = visit.location
                }
                distance += previousLocation.calcEuclideanDistanceTo(vehicle.homeLocation)
                (distance * 1_000_000.0).toLong()
            })
            .asConstraint("vehicle-routing", "Minimize distance")
    }
}

In [4]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore

@PlanningSolution
data class Schedule(
    val name: String,
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>
) {
    @PlanningScore
    var score: HardSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this("", emptyList(), emptyList())
}